# Advent of Code 2024

## Day 1

### Part 1

In [8]:
with open('inputs/day1.txt') as f:
    s = f.read().split('\n')[:-1]

left = sorted([int(line.split('   ')[0]) for line in s])
right = sorted([int(line.split('   ')[1]) for line in s])

sum([abs(left[i] - right[i]) for i in range(len(left))])

2344935

### Part 2

In [10]:
frequency = {n: right.count(n) for n in left}

sum([n*frequency[n] for n in frequency])

27647262

## Day 2

### Part 1

In [17]:
with open('inputs/day2.txt') as f:
    s = f.read().split('\n')[:-1]


def is_safe(levels):
    direction = levels[1] - levels[0]
    for i in range(1, len(levels)):
        d = levels[i] - levels[i-1]
        if direction * d < 0 or abs(d) < 1 or abs(d) > 3:
            return False
    return True


report = [[int(level) for level in line.split(' ')] for line in s]
sum([1 if is_safe(levels) else 0 for levels in report])

516

### Part 2

In [19]:
def remove_level(levels, i):
    return levels[:i] + levels[i+1:]


def is_safe_tolerate(levels):
    if is_safe(levels):
        return True
    for i in range(len(levels)):
        if is_safe(remove_level(levels, i)):
            return True
    return False


sum([1 if is_safe_tolerate(levels) else 0 for levels in report])

561

## Day 3

### Part 1

In [31]:
import re

with open('inputs/day3.txt') as f:
    s = f.read()[:-1]


def mul(command):
    num = [int(i) for i in command[4:-1].split(',')]
    return num[0]*num[1]


reg = re.compile(r'mul\(\d{1,3},\d{1,3}\)')
sum([mul(i) for i in reg.findall(s)])

189527826

### Part 2

In [36]:
reg = re.compile(r'mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)')

a = 0
enabled = True
for comm in reg.findall(s):
    if comm == 'do()':
        enabled = True
    elif comm == 'don\'t()':
        enabled = False
    elif enabled:
        a += mul(comm)

a

63013756

## Day 4

### Part 1

In [23]:
with open('inputs/day4.txt') as f:
    s = f.read().split('\n')[:-1]


def transpose(s):
    return [''.join([s[y][x] for y in range(len(s))]) for x in range(len(s[0]))]


def diagonal_words(s, min_l):
    words = []
    for y in range(len(s)):
        for x in range(len(s[0])):
            for (dx,dy) in [(1,1),(1,-1),(-1,1),(-1,-1)]:
                if x + (min_l-1)*dx in range(len(s[0])) and y + (min_l-1)*dy in range(len(s)):
                    words.append(''.join([s[y+i*dy][x+i*dx] for i in range(min_l)]))
    return words


search = 'XMAS'
a = 0

for line in s:
    a += line.count(search)
    a += line.count(search[::-1])

for line in transpose(s):
    a += line.count(search)
    a += line.count(search[::-1])

a += diagonal_words(s, len(search)).count(search)

a

2583

### Part 2

In [24]:
search = 'MAS'

def check_cross_word(x,y,s):
    x_in_range = x in range(1, len(s[0])-1)
    y_in_range = y in range(1, len(s)-1)
    if not x_in_range or not y_in_range:
        return False
    center_ok = s[y][x] == search[1]
    diag_up_left    = s[y-1][x-1] == search[0] and s[y+1][x+1] == search[2]
    diag_up_right   = s[y-1][x+1] == search[0] and s[y+1][x-1] == search[2]
    diag_down_left  = s[y+1][x-1] == search[0] and s[y-1][x+1] == search[2]
    diag_down_right = s[y+1][x+1] == search[0] and s[y-1][x-1] == search[2]
    return center_ok and (diag_up_left or diag_down_right) and (diag_up_right or diag_down_left)


len([(x,y) for y in range(len(s)) for x in range(len(s[0])) if check_cross_word(x,y,s)])

1978

## Day 5

### Part 1

In [27]:
with open('inputs/day5.txt') as f:
    s = f.read()[:-1].split('\n\n')


def in_order(pages, rules):
    visited = set()
    for page in pages:
        visited.add(page)
        related_rules = rules.get(page, [])
        for rule in related_rules:
            if rule in pages and rule not in visited:
                return False
    return True


def middle(pages):
    return pages[len(pages)//2]


rules = {}
for rule in s[0].split('\n'):
    (x,y) = tuple(int(i) for i in rule.split('|'))
    preceding = rules.get(y, [])
    preceding.append(x)
    rules[y] = preceding

pages = [[int(i) for i in line.split(',')] for line in s[1].split('\n')]

sum([middle(p) for p in pages if in_order(p, rules)])

7365

### Part 2

In [28]:
def filtered_rules(page, pages, rules, visited):
    return [r for r in rules.get(page, []) if r in pages and r not in visited]


def order(pages, rules):
    result = []
    while len(result) < len(pages):
        relevant_rules = {p: filtered_rules(p, pages, rules, result) for p in pages if p not in result}
        next_page = [r for r in relevant_rules if len(relevant_rules[r]) == 0][0]
        result.append(next_page)
    return result

sum([middle(order(p, rules)) for p in pages if not in_order(p, rules)])

5770